<a href="https://colab.research.google.com/github/smrutipunto/DS-for-argri/blob/main/DS_for_Agriculture_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PRACTICAL 1 Web Scraping for Agricultural Data: Scrape weather data, soil data, and crop yield data from relevant websites - Tools: Python, BeautifulSoup, Scrapy.
#Cleaning and Preprocessing Data: Handle missing values, outliers, and normalize data for further analysis. Exploratory Analysis of Crop Yield Data: Perform descriptive statistics and visualize data distributions, trends, and correlations
from bs4 import BeautifulSoup
import requests
d="https://en.wikipedia.org/wiki/Crop"
r=requests.get(d)
s=BeautifulSoup(r.text,'html')
print(s)
t=s.find('table',class_='nowraplinks hlist mw-collapsible autocollapse navbox-inner')
print(t)
crop_cols=s.find_all('th')
crop_cols
crop_table_cols=[t.text.strip() for t in crop_cols]
print(crop_table_cols)
import pandas as pd
df=pd.DataFrame(columns=crop_table_cols)
df
column_data=s.find_all('tr')
column_data
for row in column_data:
    row_data=row.find_all('td')
    individual_row_data=[data.text.strip for data in row_data]
    print(individual_row_data)
    for row in column_data[1:]:
    row_data=row.find_all('td')

    #print(row_data)
    individual_row_data=[data.text for data in row_data]
    length=len(df)
    df.loc[length]=individual_row_data
#individual_row_data
    #length=len(df)
    #df.loc[length]=individual_row_data
    df
    df.to_csv(r'C:\Shubhangi\2024 25\UDCS\practicals\pract 1\agri.csv',index=False)

In [ ]:
#Practical 2: Weather Prediction Using Time Series Analysis: Predict future weather patterns using historical weather data
#practical 2
> install.packages("forecast")
> library(forecast)
> data1 <- read.csv(file.choose(),header=T)
> data1
> data1 <- read.csv(file.choose(),header=T)
> data1
> fit<-auto.arima(data1)
> summary(fit)
> forecasted_values <- forecast(fit, h = 12)
> print(forecasted_values)
 Point Forecast Lo 80 Hi 80 Lo 95 Hi 95
 > plot(forecasted_values, main = "ARIMA for rainfall prediction")


In [ ]:
# Crop Yield Prediction Using Machine Learning: Build a machine learning model to predict crop yields based on various factors (soil type, weather, etc.). Tools: Scikit-learn, XGBoost
#practical 3
import numpy as np
import pandas as pd
df_yield = pd.read_csv('../input/crop-yield-predictiondataset/yield.csv')
df_yield.shape
(56717, 12)
df_yield.head()
# rename columns.
df_yield = df_yield.rename(index=str, columns={"Value":
"hg/ha_yield"})
# drop unwanted columns.
df_yield = df_yield.drop(['Year Code','Element Code','Element','Year
Code','Area Code','Domain Code','Domain','Unit','Item Code'],
axis=1)
df_yield.head()
df_yield.describe()
df_yield.info()
Index: 56717 entries, 0 to 56716
Data columns (total 4 columns):
f_rain = pd.read_csv('../input/crop-yield-predictiondataset/rainfall.csv')
df_rain.head()
df_rain['average_rain_fall_mm_per_year'] =
pd.to_numeric(df_rain['average_rain_fall_mm_per_year'],errors =
'coerce')
df_rain.info()
df_rain = df_rain.dropna()
yield_df = pd.merge(df_yield, df_rain, on=['Year','Area'])
yield_df.head()
yield_df.describe()
df_pes = pd.read_csv('../input/crop-yield-predictiondataset/pesticides.csv')
13
df_pes = df_pes.rename(index=str, columns={"Value":
"pesticides_tonnes"})
df_pes = df_pes.drop(['Element','Domain','Unit','Item'], axis=1)
df_pes.head()
yield_df = pd.merge(yield_df, df_pes, on=['Year','Area'])
yield_df.shape
AVERAGE Temperature
avg_temp= pd.read_csv('../input/crop-yield-predictiondataset/temp.csv')
avg_temp = avg_temp.rename(index=str, columns={"year": "Year",
"country":'Area'})
avg_temp.head()
yield_df = pd.merge(yield_df,avg_temp, on=['Area','Year'])
yield_df.head()
yield_df.describe()
yield_df.groupby('Item').count()
yield_df.describe()
yield_df.groupby(['Area'],sort=True)['hg/ha_yield'].sum().nlargest(1
0)
yield_df.groupby(['Item','Area'],sort=True)['hg/ha_yield'].sum().nla
rgest(10)
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
correlation_data=yield_df.select_dtypes(include=[np.number]).corr()
mask = np.zeros_like(correlation_data, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.palette="vlag"
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_data, mask=mask, cmap=cmap, vmax=.3,
center=0,
 square=True, linewidths=.5, cbar_kws={"shrink": .5});
 #DATA PREPROCESSING
from sklearn.preprocessing import OneHotEncoder
yield_df_onehot = pd.get_dummies(yield_df, columns=['Area',"Item"],
prefix = ['Country',"Item"])
features=yield_df_onehot.loc[:, yield_df_onehot.columns !=
'hg/ha_yield']
label=yield_df['hg/ha_yield']
features = features.drop(['Year'], axis=1)
#SCALING FEATURES
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
features=scaler.fit_transform(features)
#TRAINING DATA
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels =
train_test_split(features, label, test_size=0.2, random_state=42)
#TESTING DATA
#setting test data to columns from dataframe and excluding
'hg/ha_yield' values where ML model should be predicting
test_df=pd.DataFrame(test_data,columns=yield_df_onehot.loc[:,
yield_df_onehot.columns != 'hg/ha_yield'].columns)
# using stack function to return a reshaped DataFrame by pivoting
the columns of the current dataframe
cntry=test_df[[col for col in test_df.columns if 'Country' in
col]].stack()[test_df[[col for col in test_df.columns if 'Country'
in col]].stack()>0]
cntrylist=list(pd.DataFrame(cntry).index.get_level_values(1))
countries=[i.split("_")[1] for i in cntrylist]
itm=test_df[[col for col in test_df.columns if 'Item' in
col]].stack()[test_df[[col for col in test_df.columns if 'Item' in
col]].stack()>0]
itmlist=list(pd.DataFrame(itm).index.get_level_values(1))
items=[i.split("_")[1] for i in itmlist]
test_df.drop([col for col in test_df.columns if 'Item' in
col],axis=1,inplace=True)
test_df.drop([col for col in test_df.columns if 'Country' in
col],axis=1,inplace=True)
test_df.head()